In [1]:
pip install google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import storage

In [4]:
# Path to your service account JSON file
service_account_path = 'env/sodium-lodge-462105-a5-8bd3b06c45b2.json'

In [5]:
# Initialize the client
client = storage.Client.from_service_account_json(service_account_path)

In [6]:
# Your bucket and blob (file)
bucket_name = 'test-bucket-90'

In [10]:
file_name = 'Data\Harshit.JPG'

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\harsh\AppData\Local\Temp\ipykernel_12928\2847877273.py:1: SyntaxWarning: invalid escape sequence '\H'
  file_name = 'Data\Harshit.JPG'


In [11]:
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download file contents
# data = blob.download_as_text()  # Or .download_as_string()
# print(data)

In [12]:
# #  I want to save the data to a local file
# local_file_path = 'downloaded_workshop_data.json'
# with open(local_file_path, 'w') as local_file:
#     local_file.write(data)
# print(f"Data downloaded and saved to {local_file_path}")

In [13]:
# Upload a local file to the bucket
local_file_to_upload = file_name
blob_upload = bucket.blob(local_file_to_upload)
blob_upload.upload_from_filename(local_file_to_upload)
print(f"File {local_file_to_upload} uploaded to {blob_upload.name} in bucket {bucket_name}.")

File Data\Harshit.JPG uploaded to Data\Harshit.JPG in bucket test-bucket-90.


In [16]:
# List all blobs in the bucket
blobs = bucket.list_blobs()
print("Listing all blobs in the bucket:")
for blob in blobs:
    print(blob.name)

Listing all blobs in the bucket:
JSON-Data/
JSON-Data/downloaded_workshop_data.json
JSON-Data/workshop-data.json
Videos/Anaconda setup new Env.mp4
Videos/GitHub/01 setup-githubnew- account.mp4
Videos/GitHub/02 download-git-bash.mp4
Videos/GitHub/03 setup-new-repo-and-remote-setup.mp4
Videos/GitHub/04 how-to-push-remote_to_online.mp4
Videos/GitHub/05 Git-clone-demo.mp4
Videos/Python/01 Python_install & add to path.mp4


In [14]:
# Delete a blob from the bucket
blob_to_delete = 'JSON-Data/' + local_file_to_upload
blob_delete = bucket.blob(blob_to_delete)
blob_delete.delete()
print(f"Blob {blob_to_delete} deleted from bucket {bucket_name}.")

Blob JSON-Data/downloaded_workshop_data.json deleted from bucket aip-json-demo-01.


In [17]:
# Check if the blob exists
blob_exists = bucket.blob(blob_to_delete).exists()
if blob_exists:
    print(f"Blob {blob_to_delete} still exists in bucket {bucket_name}.")
else:
    print(f"Blob {blob_to_delete} has been successfully deleted from bucket {bucket_name}.")

Blob JSON-Data/downloaded_workshop_data.json still exists in bucket aip-json-demo-01.


In [18]:
# List all buckets in the project
buckets = client.list_buckets()
print("Listing all buckets in the project:")
for bucket in buckets:
    print(bucket.name)

Listing all buckets in the project:
aip-json-demo-01


In [20]:
print(f"JSON-Data/{local_file_path}")

JSON-Data/downloaded_workshop_data.json


In [ ]:
# I want to edit the downloaded JSON data and upload it back
import json
# Load the downloaded JSON data
with open(local_file_path, 'r') as local_file:
    json_data = json.load(local_file)
# Modify the JSON data (example: adding a new key-value pair)
json_data['new_key'] = 'new_value'
# Save the modified JSON data back to the local file
with open(local_file_path, 'w') as local_file:
    json.dump(json_data, local_file, indent=4)
# Upload the modified JSON data back to the bucket
blob_upload.upload_from_filename(local_file_path)
print(f"Modified file {local_file_path} uploaded back to {blob_upload.name} in bucket {bucket_name}.")
# Clean up: delete the local file after upload
import os
if os.path.exists(local_file_path):
    os.remove(local_file_path)
    print(f"Local file {local_file_path} deleted after upload.")
# End of script

In [21]:
#  if I want to upload the entire directory to the bucket
import os
def upload_directory_to_bucket(bucket, source_dir, destination_dir):
    '''Uploads an entire directory to a Google Cloud Storage bucket.'''
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            local_file_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_file_path, source_dir)
            blob_path = os.path.join(destination_dir, relative_path).replace("\\", "/")
            blob = bucket.blob(blob_path)
            blob.upload_from_filename(local_file_path)
            print(f"Uploaded {local_file_path} to {blob.name}")

In [ ]:
# Specify the directory to upload
source_directory = 'Basic Setup'  # Change this to your directory path
destination_directory = 'Videos/'  # Change this to your desired destination in the bucket
# upload_directory_to_bucket(bucket, source_directory, destination_directory)

upload_directory_to_bucket(bucket, source_directory, destination_directory)
print(f"All files from {source_directory} uploaded to {destination_directory} in bucket {bucket_name}.")

All files from Basic Setup uploaded to Videos/ in bucket aip-json-demo-01.


In [ ]:
# List all blobs in the destination directory
print(f"Listing all blobs in the directory {destination_directory}:")
blobs_in_directory = bucket.list_blobs(prefix=destination_directory + '/')
for blob in blobs_in_directory:
    print(blob.name)

Listing all blobs in the directory Videos/:


In [ ]:
# Clean up: delete the uploaded directory blobs
for blob in blobs_in_directory:
    blob.delete()
    print(f"Blob {blob.name} deleted from bucket {bucket_name}.")
# End of script